In [ ]:
from google.colab import drive
drive.mount('/content/drive/', force_remount=True)

Mounted at /content/drive/


In [ ]:
!pip -q install tensorflow tensorflow-gpu opencv-python matplotlib keras

  error: subprocess-exited-with-error
  
  × python setup.py egg_info did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  Preparing metadata (setup.py) ... error
error: metadata-generation-failed

× Encountered error while generating package metadata.
╰─> See above for output.

note: This is an issue with the package mentioned above, not pip.
hint: See above for details.


In [ ]:
import os
import tensorflow as tf
from tensorflow import keras
import pandas as pd
from keras import layers
import matplotlib
import pathlib
import PIL as Image
import imghdr
import time
from matplotlib import pyplot as plt
import numpy as np
from pathlib import Path
from keras.models import Sequential, load_model # MAY NEED TO BE REPLACED W/ FUNCTIONAL API
from keras.layers import Conv2D, MaxPool2D, Dense, Flatten, Dropout, AveragePooling2D
from keras.metrics import Precision, Recall, BinaryAccuracy

In [ ]:
# !ls 'drive/MyDrive/Colab Notebooks/Polyamanita/data/train/'

In [ ]:
model_name = 'gdemo_FINAL3'
model_folder = 'drive/MyDrive/Colab Notebooks/Polyamanita/models/' + model_name
data_folder = 'drive/MyDrive/Colab Notebooks/Polyamanita/data/train_gdemo'

In [ ]:
size = 512
batch_size = 16

In [ ]:
def expand2square(pil_img, background_color):
    width, height = pil_img.size
    if width == height:
        return pil_img
    elif width > height:
        result = Image.new(pil_img.mode, (width, width), background_color)
        result.paste(pil_img, (0, (width - height) // 2))
        return result
    else:
        result = Image.new(pil_img.mode, (height, height), background_color)
        result.paste(pil_img, ((height - width) // 2, 0))
        return result

In [ ]:
model = tf.keras.models.load_model(model_folder + '/savedmodel.h5', compile = True) # Load the model from the .h5 file
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 512, 512, 3)]     0         
                                                                 
 sequential (Sequential)     (None, 512, 512, 3)       0         
                                                                 
 tf.math.truediv (TFOpLambda  (None, 512, 512, 3)      0         
 )                                                               
                                                                 
 tf.math.subtract (TFOpLambd  (None, 512, 512, 3)      0         
 a)                                                              
                                                                 
 mobilenetv2_1.00_224 (Funct  (None, 16, 16, 1280)     2257984   
 ional)                                                          
                                                             

In [ ]:
dataset = tf.keras.utils.image_dataset_from_directory(
    data_folder,
    shuffle = False,
    image_size = (size,size),
    batch_size = batch_size,
    labels='inferred',
    label_mode = 'categorical'
)

class_names = dataset.class_names

Found 11294 files belonging to 16 classes.


In [ ]:
import joblib

In [ ]:
dir_order = os.listdir(data_folder)
Y = []
Y_links = []
for dir in dir_order:
    jpgs = os.listdir(data_folder + '/' + dir)
    num_files = len(jpgs)
    for i in range(num_files):
        Y.append(dir)
        Y_links.append(dir + '/' + jpgs[i])

# print(Y)
# print(Y_links)

In [ ]:
prediction = []
print(data_folder + '/' + Y_links[0])
for link in Y_links:
    try:
        img = tf.keras.preprocessing.image.load_img(data_folder + '/' + link, target_size=(size, size, 3))
        sq_img = expand2square(img, (0,0,0))
        sq_img = np.reshape(img,[1,size,size,3])
        img_pred = model.predict(sq_img, verbose = 0)
        prediction.append(img_pred) # THIS TAKES A WHILE # :( -jan
    
    except Exception as e:
        print(f'Bad: {e}\n{link}')
        break

prediction = np.array(prediction)

# def preprocess_image(filename, size):
#     img = tf.keras.preprocessing.image.load_img(filename, target_size=(size, size, 3))
#     sq_img = expand2square(img, (0,0,0))
#     sq_img = np.reshape(sq_img, [1,size,size,3])
#     return sq_img

# batches = [Y_links[i:i+batch_size] for i in range(0, len(Y_links), batch_size)]
# for batch in batches:
#     preprocessed_images = joblib.Parallel(n_jobs=-1)(
#         joblib.delayed(preprocess_image)(data_folder + '/' + filename, size) for filename in batch)
#     batch_predictions = model.predict(np.concatenate(preprocessed_images), verbose=0)
#     prediction.extend(batch_predictions)

print(prediction.shape)

drive/MyDrive/Colab Notebooks/Polyamanita/data/train_gdemo/Lycoperdon_marginatum/Lycoperdon_marginatum_271.jpg


AttributeError: ignored

In [ ]:
prediction = prediction.reshape(prediction.shape[0], prediction.shape[2])
sig = tf.nn.sigmoid(prediction)
guess = tf.nn.softmax(sig)

n = guess.shape[0]
pred_class = np.zeros(n)
guessed_name = []

print(guess.shape)
# print(np.argmax(guess[-5:]))

# for i in range(n):
#     pred_class[i] = np.argmax(guess[i])
# pred_idx = np.argmax(guess, axis=1)

# for i in range(n):
#     print(i, ': ', class_names[pred_idx[i]])

In [ ]:
np.save(model_folder + '/pred_temp', guess)

In [ ]:
predictions = np.load(model_folder + '/pred_temp.npy')

In [ ]:
classes = dataset.class_names
n = predictions.shape[0]

Y_pred_idx = np.argmax(predictions, axis=1)
Y_pred = []

for i in range(n):
    Y_pred.append(classes[Y_pred_idx[i]])
    print(i, ':', Y[i], '--', Y_pred[i])

In [ ]:
results = pd.DataFrame({'Y': Y, 'Y_pred': Y_pred})
results['match'] = (results['Y'] == results['Y_pred']) # Generate matching column
print(results)

In [ ]:
total_correct = results['match'].sum()
print('Correct:\t', total_correct, '\nIncorrect:\t', results.shape[0] - total_correct)
accuracy = (results['match'].sum() / results.shape[0]) * 100
print(f'Accuracy:\t {round(accuracy, 3)}%')

In [ ]:
class_results = pd.DataFrame({'class': classes})
class_results['total_correct'] = 0
class_results['total_incorrect'] = 0
class_results['total'] = 0
# print(class_results)

In [ ]:
for i in range(results.shape[0]):
    class_name = results['Y'].loc[i]
    row_idx = class_results.loc[class_results['class'] == class_name].index
    if results.loc[i, 'match']:
        class_results.loc[row_idx, 'total_correct'] += 1
    else:
        class_results.loc[row_idx, 'total_incorrect'] += 1

    class_results.loc[row_idx, 'total'] += 1

In [ ]:
class_results['accuracy'] = (class_results['total_correct'] / class_results['total']) * 100
print(class_results)

In [ ]:
class_results.to_csv(model_folder + '/class_analysis.csv')

In [ ]:
print('loll')

In [ ]:
print('well  well  well')